In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
import json
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/ai_diary/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.getenv("MODEL_NAME")

'llama-3.3-70b-versatile'

In [3]:
client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url=os.getenv("GROQ_BASE_URL"),
)

model_name = os.getenv("MODEL_NAME")

def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({
            "role": message["role"],
            "content": message["content"]
        })

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2048,
    ).choices[0].message.content

    return response

In [4]:
messages = [{'role': 'user', 'content': "What's the capital of Italy?"}]
response = get_chatbot_response(client, model_name, messages)
print("Response:", response)

Response: The capital of Italy is Rome.


# Prompt Engineering

### Structured Output

In [6]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. you are not allowed to write anything other than the json object:
[
{
   "country": the country that you will get the capital of
   "capital": the capital of the country stated
}
]
"""

messages = [{"role": "system" , "content":system_prompt}]
messages.append({"role":"user" , "content":"What's the capital of Germany?"})
response = get_chatbot_response(client=client , model_name=model_name , messages=messages)
print(response)


[
{
   "country": "Germany",
   "capital": "Berlin"
}
]


In [7]:
type(response)

str

In [9]:
import json

In [10]:
json_response = json.loads(response)
json_response

[{'country': 'Germany', 'capital': 'Berlin'}]

In [11]:
type(json_response[0]) , json_response[0]["capital"]


(dict, 'Berlin')

### Input Structure


In [13]:
user_input = """
Get me the capitals of the following countries:
'''
1. Italy
2. Germany
3. France
''' 
"""

messages = [{"role": "system" , "content": system_prompt}]
messages.append({"role":"user" , "content": user_input})
response = get_chatbot_response(client , model_name=model_name , messages=messages)

In [15]:
print(response)

[
{
   "country": "Italy",
   "capital": "Rome"
},
{
   "country": "Germany",
   "capital": "Berlin"
},
{
   "country": "France",
   "capital": "Paris"
}
]


In [16]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'France', 'capital': 'Paris'}]

### Give the model time to think (Chain of Thought)

In [17]:
user_prompt = """ 
Calculate the result of this equation: 1 + 3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
   result: The final number resulted from calculating the equation above
}
"""

messages = [{"role": "user" , "content": user_prompt}]
response = get_chatbot_response(client , model_name=model_name , messages=messages)
print(response)

{
   "result": 4
}


In [18]:
259/2*8654+91072*33-12971

4113098.0

In [22]:
user_prompt = """ 
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
   result: The final number resulted from calculating the equation above
}
"""

messages = [{"role": "user" , "content": user_prompt}]
response = get_chatbot_response(client , model_name=model_name , messages=messages)
print(response)

{
   "result": 259/2*8654+91072*33-12971
}


In [24]:
user_prompt = """ 
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{  steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final result.Feel free to write in free text.
   result: The final number resulted from calculating the equation above
}
"""


messages = [{"role": "user" , "content": user_prompt}]
response = get_chatbot_response(client , model_name=model_name , messages=messages)

print(response)

{
  steps: 
    To calculate the given equation, we need to follow the BEDMAS (Brackets, Exponents, Division, Multiplication, Addition, and Subtraction) order of operations. 
    The equation is: 259/2*8654+91072*33-12971.
    First, we do the division: 259/2 = 129.5.
    Then, we do the multiplication: 129.5*8654 = 11221330 and 91072*33 = 3006376.
    Now, the equation becomes: 11221330 + 3006376 - 12971.
    Next, we do the addition and subtraction from left to right: 11221330 + 3006376 = 14227706, then 14227706 - 12971 = 14214735.
    Therefore, the final result of the equation is 14214735.
  ,
  result: 14214735
}


# RAG : Retrieval Augmented Generation

In [26]:
user_prompt = """
What's new in iphone 16? 
"""

messages = [{"role":"user" , "content":user_prompt}]
response = get_chatbot_response(client , model_name=model_name , messages=messages)
print(response)

Since I'm a large language model, my knowledge cutoff is December 2023, and I don't have real-time information. As of my knowledge cutoff, there was no official announcement from Apple about the iPhone 16.

However, I can provide you with some rumored features and specifications that were circulating online:

1. **Design**: The iPhone 16 is expected to have a similar design to the iPhone 14 series, with some minor tweaks.
2. **Display**: The new iPhone might feature a higher refresh rate, possibly up to 240Hz, and improved OLED panels.
3. **Cameras**: Rumors suggest that the iPhone 16 could have a periscope lens with a 5x or 6x optical zoom, improved portrait mode, and enhanced low-light performance.
4. **Processors**: The iPhone 16 is likely to be powered by Apple's A18 Bionic chip, which could provide significant performance and efficiency improvements.
5. **Battery Life**: The new iPhone might have a larger battery, possibly up to 4500mAh, and improved power management for longer ba

In [27]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [28]:
user_prompt = f"""
{iphone_16}
What's new in iphone 16?
"""

messages = [{"role":"user" , "content":user_prompt}]
response = get_chatbot_response(client , model_name , messages)
print(response)

The iPhone 16 introduces several new features, including:

1. **Larger displays**: The base model has a 6.1-inch display, while the iPhone 16 Plus has a 6.7-inch screen. The Pro models have even larger displays, measuring 6.3 and 6.9 inches.
2. **Thinner bezels**: The iPhone 16 has thinner bezels, with the Pro models having the thinnest bezels seen on any Apple product.
3. **More durable Ceramic Shield**: The display is more durable and resistant to damage.
4. **A18 chip (A18 Pro for Pro models)**: Significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks.
5. **Upgraded camera systems**: The base iPhone 16 has a dual-camera setup with a 48MP main sensor, while the Pro models have a 48MP Ultra Wide and 5x telephoto camera, with a "Camera Control" button for more flexible photography options.
6. **Advanced audio features**: "Audio Mix" uses machine learning to separate background sounds from speech, allowing for more

### Automatically Extract context data from database

In [29]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""


In [30]:
data = [iphone_16 , samsung_s23]

In [46]:
user_prompt = "What's new in samsumg s23?"

In [48]:
model = SentenceTransformer("hkunlp/instructor-xl")
sentences = ["What is the best embedding model?", "Llama 3.2 is great!"]
embeddings = model.encode(sentences)
print(embeddings.shape)  # Output shape: (2, 768)

RuntimeError: MPS backend out of memory (MPS allocated: 9.06 GB, other allocations: 704.00 KB, max allowed: 9.07 GB). Tried to allocate 125.50 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [49]:
user_prompt_embeddings = model.encode(user_prompt)

RuntimeError: MPS backend out of memory (MPS allocated: 9.07 GB, other allocations: 704.00 KB, max allowed: 9.07 GB). Tried to allocate 256 bytes on shared pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [50]:
import numpy as np

def get_embedding(model, text_input):
    """
    Generate an embedding similar to OpenAI's structure.
    Ensures output is a 2D array.
    """
    embeddings = model.encode(text_input)

    # Ensure the output is a 2D array (1 sample, N features)
    return np.array(embeddings).reshape(1, -1)

In [51]:
data_embeddings = [get_embedding(model , text)[0] for text in data]

RuntimeError: MPS backend out of memory (MPS allocated: 9.06 GB, other allocations: 8.69 MB, max allowed: 9.07 GB). Tried to allocate 2.75 KB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [52]:
len(data_embeddings)

2

In [53]:
data_similarity = cosine_similarity([user_prompt_embeddings] , data_embeddings)

In [54]:
data_similarity

array([[0.9985357, 0.786505 ]], dtype=float32)

In [55]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

0

In [56]:
data[closest_entry_index]


'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [57]:
user_prompt_with_data = f""" 
{data[closest_entry_index]}

{user_prompt}
"""

In [58]:
print(user_prompt_with_data)

 

The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learni

In [ ]:
messages = [{"role":"user" , "content":user_prompt_with_data}]
response = get_chatbot_response(client , model_name , messages)
print(response)

The iPhone 16 introduces several new features, including:

1. **Larger displays**: The base model has a 6.1-inch display, the iPhone 16 Plus has a 6.7-inch screen, and the Pro models have 6.3-inch and 6.9-inch displays.
2. **Thinner bezels**: The iPhone 16 Pro and Pro Max have the thinnest bezels seen on any Apple product.
3. **More durable Ceramic Shield**: The displays are protected by a more durable Ceramic Shield.
4. **New A18 chip (A18 Pro for Pro models)**: Delivering significant performance improvements, enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks.
5. **Upgraded camera systems**: The base iPhone 16 has a dual-camera setup with a 48MP main sensor, while the Pro models offer a 48MP Ultra Wide and 5x telephoto camera.
6. **"Camera Control" button**: For more flexible photography options on the Pro models.
7. **Advanced audio features**: "Audio Mix" uses machine learning to separate background sounds from speech, allowing for more refined a